**BBM473 Database Laboratory (Spring 2022)**


Exercise 3: More on SQL
-----

Please run the below cells first before proceeding- you'll need them soon!

In [ ]:
%load_ext sql
%sql sqlite://

'Connected: @None'

In [ ]:
%%sql 
DROP TABLE IF EXISTS Movies;
CREATE TABLE Movies(title VARCHAR(50), year INT, director VARCHAR(50), length INT);
INSERT INTO Movies VALUES('Database Wars', 1967, 'John Joe', 123);
INSERT INTO Movies VALUES('The Databaser', 1992, 'John Bob', 190);
INSERT INTO Movies VALUES('Database Wars', 1998, 'John Jim', 176);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [ ]:
%sql DROP TABLE IF EXISTS A; DROP TABLE IF EXISTS B;
%sql CREATE TABLE A (x int, y int); CREATE TABLE B (x int, y int);
for i in range(1,6):
    %sql INSERT INTO A VALUES (:i, :i+1)
for i in range(1,11,3):
    %sql INSERT INTO B VALUES (:i, :i+2)

 * sqlite://
Done.
Done.
 * sqlite://
Done.
Done.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.
 * sqlite://
1 rows affected.


Activity 3-1:
------------

ORDER BY semantics, set operators & nested queries

In [ ]:
%sql SELECT * FROM movies

   sqlite://
 * sqlite:///dataset_1.db
Done.


title,year,director,length
Database Wars,1967,John Joe,123
The Databaser,1992,John Bob,190
Database Wars,1998,John Jim,176


Task #1 (15 pts.)
-----------

**Can you write the movie query below as a single SFW query?**

Recall that we are trying to find **all movie titles that were used for more than one movie.** You may assume that no two movies in the same year have the same title. Our schema for the `movies` table is:

> * title STRING
> * year INT
> * director STRING
> * length INT

Let's try to write the nested query that solves this:

In [ ]:
%%sql
SELECT m.title 
FROM Movies m
WHERE m.year <> ANY(SELECT year FROM Movies WHERE title = m.title);

 * sqlite://
(sqlite3.OperationalError) near "SELECT": syntax error
[SQL: SELECT m.title 
FROM Movies m
WHERE m.year <> ANY(SELECT year FROM Movies WHERE title = m.title);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


**What**?  This doesn't work?  Why?

**ANY doesn't exist in SQLite!**  Can we do this query without nesting?  Write your query here:

In [ ]:
%%sql
-- Note: It works but it's still a nested query, so solution NOT COMPLETED
SELECT m.title 
FROM Movies m
WHERE m.year in (SELECT year FROM Movies WHERE title = m.title);

   sqlite://
 * sqlite:///dataset_1.db
Done.


title
Database Wars
The Databaser
Database Wars


Task #2 (15 pts.)
--------------------

Consider the two relations $A$ and $B$ below:

In [ ]:
%sql SELECT * FROM A;

 * sqlite://
Done.


x,y
1,2
2,3
3,4
4,5
5,6


In [ ]:
%sql SELECT * FROM B;

 * sqlite://
Done.


x,y
1,3
4,6
7,9
10,12


Assuming no duplicates, can you write an `INTERSECT` query, **just over the $x$ attribute**, without using `INTERSECT` OR nested queries?  Write your query here:

In [ ]:
%%sql
SELECT A.x as "x", A.y as "y_of_a", B.y as "y_of_b" FROM A JOIN B
ON A.x = B.x

 * sqlite://
Done.


x,y_of_a,y_of_b
1,2,3
4,5,6


What is this operation called?

Next, using set operators again as well, can you return all the _full_ tuples in $A$ and $B$ that overlap in $x$ attributes?  Write your query here:

In [ ]:
%%sql
SELECT x FROM A INTERSECT SELECT x FROM B
-- Note: Not all tuples returned, so solution NOT COMPLETED

 * sqlite://
(sqlite3.OperationalError) SELECTs to the left and right of INTERSECT do not have the same number of result columns
[SQL: SELECT x, A.y FROM A INTERSECT SELECT x FROM B]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Execute the code below:

In [ ]:
%sql sqlite:///dataset_1.db

'Connected: @dataset_1.db'

Activity 3-2
------------
Aggregation operators, GROUP BY

Task #1 (20 pts.)
-----------

Consider a set of tables that describe the up-and-coming bagel startup industry; for now let's just look at two tables here, `bagel`, which describes types of bagels made by the different bagel companies:
> * name STRING
> * price FLOAT
> * made_by STRING

And `purchase`:
> * bagel_name STRING
> * franchise STRING
> * date INT
> * quantity INT
> * purchaser_age INT

Where `purchase.bagel_name` references `bagel.name` and `purchase.franchise` references `bagel.made_by`:

In [ ]:
%sql SELECT * FROM bagel LIMIT 3;

   sqlite://
 * sqlite:///dataset_1.db
Done.


name,price,made_by
Plain with shmear,1.99,Bobs Bagels
Egg with shmear,2.39,Bobs Bagels
eBagel Drinkable Bagel,27.99,eBagel


In [ ]:
%sql SELECT * FROM purchase LIMIT 3;

   sqlite://
 * sqlite:///dataset_1.db
Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24


Can you write a query to get the _total revenue_ for each bagel type **which had an average purchaser age over 18**?  Type your query below:

In [ ]:
%%sql
-- Missing values in the database, may cause problems there
SELECT (quantity*price) as "total_revenue", name as "name", AVG(purchaser_age) as "avg_age" FROM bagel JOIN purchase
ON bagel.name = purchase.bagel_name
GROUP BY name
HAVING AVG(purchaser_age) > 18

   sqlite://
 * sqlite:///dataset_1.db
Done.


total_revenue,name,avg_age
14.34,Egg with shmear,47.0
23.759999999999998,Plain with shmear,23.0


Task #2 (20 pts.)
-----------

Here we'll use a simplified version of the `precipitation_full` table, which just has _daily_ rainfall _in CA only_, and has the following schema:

> * station_id
> * day
> * precipitation

In [ ]:
%sql SELECT * FROM precipitation LIMIT 5;

   sqlite://
 * sqlite:///dataset_1.db
Done.


station_id,day,precipitation
16102,1,10
16102,4,10
16102,24,30
21201,1,0
21201,20,10


We want to get station_ids which have average precipitations > 75.  Try doing this first as a nested query:

In [ ]:
%%sql 
SELECT DISTINCT p.station_id FROM precipitation p 
WHERE (
  SELECT AVG(precipitation) FROM precipitation WHERE station_id = p.station_id
) > 75;

   sqlite://
 * sqlite:///dataset_1.db
Done.


station_id
88302
250002
335701
357302
488301


Now, try re-writing as a GROUP BY:

In [ ]:
%%sql 
SELECT station_id FROM precipitation
GROUP BY station_id
HAVING AVG(precipitation)>75


   sqlite://
 * sqlite:///dataset_1.db
Done.


station_id
88302
250002
335701
357302
488301


Now time it by using `%time` followed by single-line versions of your queries above (clunky, but will work) to see how they compare!

**Note:** Yes, currently the answers are filled in below for convenience... but you should still try getting them on your own above!

In [ ]:
%%time 
%%sql 
SELECT DISTINCT p.station_id 
FROM precipitation p 
WHERE (SELECT AVG(precipitation) FROM precipitation WHERE station_id = p.station_id) > 75;

   sqlite://
 * sqlite:///dataset_1.db
Done.
CPU times: user 25.5 ms, sys: 0 ns, total: 25.5 ms
Wall time: 24.9 ms


station_id
88302
250002
335701
357302
488301


In [ ]:
%time %sql SELECT p.station_id FROM precipitation p GROUP BY p.station_id HAVING AVG(p.precipitation) > 75;

   sqlite://
 * sqlite:///dataset_1.db
Done.
CPU times: user 4.49 ms, sys: 0 ns, total: 4.49 ms
Wall time: 4.56 ms


station_id
88302
250002
335701
357302
488301


**An ~ 10-20x difference in execution time!!**

Activity 3-3
------------
Quantifiers, NULLs, and Outer Joins

Task #1 (40 pts.)
-----------

Recall that the tables we just looked at:

`bagel`, which describes types of bagels made by the different bagel companies:
> * name STRING
> * price FLOAT
> * made_by STRING

And `purchase`:
> * bagel_name STRING
> * franchise STRING
> * date INT
> * quantity INT
> * purchaser_age INT

Where `purchase.bagel_name` references `bagel.name` and `purchase.franchise` references `bagel.made_by`.

**Can you find out if there were any purchases of products not on one of the company's official lists (i.e. the `bagel` table), using a single SQL query?**

Write your query here:

In [ ]:
%%sql
SELECT bagel_name, franchise, date, quantity, purchaser_age
FROM purchase p LEFT OUTER JOIN bagel b
ON b.name = p.bagel_name AND b.made_by = p.franchise
WHERE b.name IS NULL;

   sqlite://
 * sqlite:///dataset_1.db
Done.


bagel_name,franchise,date,quantity,purchaser_age
Moonshine,BAGEL CORP,7,1000,37
